<a href="https://colab.research.google.com/github/alhichri/SSDAN/blob/main/main_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install efficientnet_pytorch
!pip install MulticoreTSNE


from __future__ import print_function
import argparse
import os
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable,function
import torch.nn.functional as F

from torchvision import transforms

In [2]:
from google.colab import drive
drive.mount('/content/gdrive' , force_remount=True)

Mounted at /content/gdrive


In [ ]:
#####################################################
# Define paths and parameters
#####################################################

main_folder_path = '/content/gdrive/MyDrive/'
SSDAN_path = '/content/gdrive/MyDrive/code/SSDAN/'

datasets_path = main_folder_path+'datasets/our_datasets/'
import os
print('Contents of folder: ', datasets_path)
os.listdir(datasets_path) # returns list

# set arguments here

args_method = 'MME'; # choices=['S+T', 'ENT', 'MME']   help='MME is proposed method, ENT is entropy minimization,'

args_epochs = 5001     # maximum number of training epochs 
args_batchsize = 8     # batch size for dividing the training data during training
args_lr = 0.001      # help='learning rate (default: 0.001)'
args_multi = 0.1      #   help='learning rate multiplication'
args_T = 0.05     #   help='temperature (default: 0.05)'
args_lamda =  0.1     #help='value of lamda parameters'
args_save_check = False  #  help='save checkpoint or not'
args_checkpath = SSDAN_path+'outputsAndPlots'   # help='dir to save checkpoint'
args_seed = 2021                    # help='random seed (default: 1)'

args_log_interval = 50          # help='how many batches to wait before logging '  'training status'
args_net  = 'EfficientNetBase'          # choices=['resnet34', 'alexnet', 'vgg' ,'EfficientNetBase']   help='which network to use'

args_dataset   =  'TWO-SOURCE'        #  choices=['THREE-SOURCE', 'TWO-SOURCE']  help='how many source datasets'
                             # we will change this to choices=['UCMerced', 'AID', 'NWPU', 'PatternNet']  help='the name of dataset'
args_source  = 'AID_NWPU_RESISC45'            #   help='source domain' we will change this to choices=['UCMerced', 'AID', 'NWPU',
                                                   # 'AID_UCMerced' ,'UCMerced_NWPU_RESISC45', 'AID_NWPU_RESISC45',
                                                   # 'AID_NWPU_RESISC45_PatternNet', 'AID_NWPU_RESISC45_UCMerced', 'AID_UCMerced_PatternNet'
                                                   #, 'UCMerced_NWPU_RESISC45_PatternNet']
args_target  = 'UCMerced'         #  help='target domain' , 'UCMerced', 'AID', 'NWPU'
args_num  = 3                    #  help='number of labeled examples in the target'
args_early  = True                  #    help='early stopping on validation or not'
args_patience  = 5               # help='early stopping to wait for improvment '  'before terminating. (default: 5 (5000 iterations))')
blocks_args = 3

print('Dataset: %s Source: %s Target: %s Labeled num perclass: %s Network: %s' %
      (args_dataset, args_source, args_target, args_num, args_net))
#######################  One source  #######################
#source_path = datasets_path+'%s/' % args_dataset  + 'NWPU_RESISC45/'
#source_path = datasets_path+'%s/' % args_dataset  + 'AID/'
#source_path = datasets_path+'%s/' % args_dataset  + 'UCMerced/'
#######################  Two source  #######################
# source_path = datasets_path+'%s/' % args_dataset  + 'AID_UCMerced/'
# source_path = datasets_path+'%s/' % args_dataset  + 'UCMerced_NWPU_RESISC45/'
source_path = datasets_path+'%s/' % args_dataset  + 'AID_NWPU_RESISC45/'
#######################  Three source  #######################
#source_path = datasets_path+'%s/' % args_dataset  + 'AID_NWPU_RESISC45_PatternNet/'
#source_path = datasets_path+'%s/' % args_dataset  + 'AID_NWPU_RESISC45_UCMerced/'
#source_path = datasets_path+'%s/' % args_dataset  + 'AID_UCMerced_PatternNet/'
#source_path = datasets_path+'%s/' % args_dataset  + 'UCMerced_NWPU_RESISC45_PatternNet/'
print('Source dataset path: ', source_path)
 
target_path = datasets_path+'%s/' % args_dataset  + 'UCMerced/'
# target_path = datasets_path+'%s/' % args_dataset  + 'NWPU_RESISC45/'
# target_path = datasets_path+'%s/' % args_dataset  + 'AID/'
#target_path = datasets_path+'%s/' % args_dataset  + 'PatternNet/'
print('Target dataset path: ', target_path)


Dataset: TWO-SOURCE Source: AID_UCMerced Target: NWPU Labeled num perclass: 3 Network: EfficientNetBase


In [7]:
from importlib.machinery import SourceFileLoader
from importlib.util import spec_from_loader, module_from_spec

somemodule = SourceFileLoader('utils', SSDAN_path+'utils/utils.py').load_module()
from utils import weights_init, efficientnet

somemodule = SourceFileLoader('loss', SSDAN_path+'utils/loss.py').load_module()
from loss import entropy, adentropy

somemodule = SourceFileLoader('lr_schedule', SSDAN_path+'utils/lr_schedule.py').load_module()
from lr_schedule import inv_lr_scheduler

#somemodule = SourceFileLoader('return_dataset', SSDAN_path+'utils/return_dataset.py').load_module()
#from return_dataset import return_dataset

somemodule = SourceFileLoader('EfficientNet', SSDAN_path+'model/EfficientNet.py').load_module()
from EfficientNet import EfficientNetBase

somemodule = SourceFileLoader('basenet', SSDAN_path+'model/basenet.py').load_module()
from basenet import AlexNetBase, VGGBase, Predictor, Predictor_deep

somemodule = SourceFileLoader('resnet', SSDAN_path+'model/resnet.py').load_module()
from resnet import resnet34

somemodule = SourceFileLoader('data_list', SSDAN_path+'loaders/data_list.py').load_module()
from data_list import Imagelists_VISDA, return_classlist


somemodule = SourceFileLoader('tsne_torch', SSDAN_path+'tsne_torch.py').load_module()
from tsne_torch import tsne




In [ ]:
################################################################
# here define the folders where the list of files is found
# these are text files that contain the list of images, including the full path,  
# to be used from each dataset .
################################################################

# list of images in source dataset
image_set_file_s = \
    os.path.join(main_folder_path,
                  'labeled_source_images_' +
                  args_source + '.txt')
# list of labeled images from target dataset (K from each class) used for training
image_set_file_t = \
    os.path.join(main_folder_path,
                  'labeled_target_images_' +
                  args_target + '_%d.txt' % (args_num))
# list of labeled images from target dataset (K from each class) used for validation and early stopping
image_set_file_t_val = \
    os.path.join(main_folder_path,
                  'validation_target_images_' +
                  args_target + '_%d.txt'% (args_num))
# list of remaining unlabeled images from target dataset 
image_set_file_unl = \
    os.path.join(main_folder_path,
                  'unlabeled_target_images_' +
                  args_target + '_%d.txt' % (args_num))
    
print('image_set_file_s: ' , image_set_file_s)
print('image_set_file_t: ' , image_set_file_t)
print('image_set_file_t_val: ' , image_set_file_t_val)
print('image_set_file_unl: ' , image_set_file_unl)

In [ ]:
################################################################
# here define the folders where th elist of files is found
# these are text files that contain the list of images, including the full path,  
# to be used from each dataset .
################################################################

crop_size = 224

class ResizeImage():
    def __init__(self, size):
        if isinstance(size, int):
            self.size = (int(size), int(size))
        else:
            self.size = size

    def __call__(self, img):
        th, tw = self.size
        return img.resize((th, tw))

data_transforms = {
    'train': transforms.Compose([
        ResizeImage(256),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(crop_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        ResizeImage(256),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(crop_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        ResizeImage(256),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print('Source dataset path: ', source_path)
source_dataset = Imagelists_VISDA(image_set_file_s, root=source_path,
                                  transform=data_transforms['train'])

print('Source dataset path: ', target_path)
target_dataset = Imagelists_VISDA(image_set_file_t, root=target_path,
                                  transform=data_transforms['val'])
target_dataset_val = Imagelists_VISDA(image_set_file_t_val, root=target_path,
                                      transform=data_transforms['val'])
target_dataset_unl = Imagelists_VISDA(image_set_file_unl, root=target_path,
                                      transform=data_transforms['val'])
target_dataset_test = Imagelists_VISDA(image_set_file_unl, root=target_path,
                                        transform=data_transforms['test'])
class_list = return_classlist(image_set_file_s)
print("%d classes in this dataset" % len(class_list))

source_loader = torch.utils.data.DataLoader(source_dataset, batch_size=args_batchsize,
                                            num_workers=2, shuffle=True,
                                            drop_last=True)
target_loader = \
    torch.utils.data.DataLoader(target_dataset,
                                batch_size=min(args_batchsize, len(target_dataset)),
                                num_workers=2,
                                shuffle=True, drop_last=True)
target_loader_val = \
    torch.utils.data.DataLoader(target_dataset_val,
                                batch_size=min(args_batchsize,
                                                len(target_dataset_val)),
                                num_workers=2,
                                shuffle=True, drop_last=True)
target_loader_unl = \
    torch.utils.data.DataLoader(target_dataset_unl,
                                batch_size=args_batchsize * 2, num_workers=2,
                                shuffle=True, drop_last=True)
target_loader_test = \
    torch.utils.data.DataLoader(target_dataset_test,
                                batch_size=args_batchsize * 2, num_workers=2,
                                shuffle=True, drop_last=True)
    

# return source_loader, target_loader, target_loader_unl, \
#     target_loader_val, target_loader_test, class_list


In [ ]:

use_gpu = torch.cuda.is_available(); print('use_gpu: ' , use_gpu)
outputsAndPlots_dir = SSDAN_path+'outputsAndPlots/%s/%s' % (args_dataset, args_method); print(outputsAndPlots_dir)
if not os.path.exists(outputsAndPlots_dir):
    os.makedirs(outputsAndPlots_dir)
outputsAndPlots_file = os.path.join(outputsAndPlots_dir,
                           '%s_net_%s_%s_to_%s_num_%s' %
                           (args_method, args_net, args_source,
                            args_target, args_num))
plot_path = outputsAndPlots_file
print('plot_path : ' , plot_path)

In [ ]:

#  model_name (str): Name for efficientnet.
#             weights_path (None or str):
#                 str: path to pretrained weights file on the local disk.
#                 None: use pretrained weights downloaded from the Internet.
#             advprop (bool):
#                 Whether to load pretrained weights
#                 trained with advprop (valid when weights_path is None).
#             in_channels (int): Input data's channel number.
#             num_classes (int):
#                 Number of categories for classification.
#                 It controls the output size for final linear layer.
#             override_params (other key word params):
#                 Params to override model's global_params.
#                 Optional key:
#                     'width_coefficient', 'depth_coefficient',
#                     'image_size', 'dropout_rate',
#                     'batch_norm_momentum',
#                     'batch_norm_epsilon', 'drop_connect_rate',
#                     'depth_divisor', 'min_depth'

# Define the generator CNN model "G", load its pre-trained parameters
# then define the predictor F
print('args_net: ' , args_net)
torch.cuda.manual_seed(args_seed)
if args_net == 'resnet34':
    G = resnet34()
    inc = 512
elif args_net == "EfficientNetBase":
    blocks_args, global_params = efficientnet(width_coefficient=None, depth_coefficient=None, image_size=(256,256,3), dropout_rate=0.2, drop_connect_rate=0.2, num_classes=1000, include_top=False)
    G = EfficientNetBase.from_pretrained(model_name='efficientnet-b3',advprop=True)
    #G = nn.Sequential(*(list(G1.children())[:-3]))
    G.eval()
    inc = 1000
elif args_net == "vgg":
    G = VGGBase()
    inc = 4096
else:
    raise ValueError('Model cannot be recognized.')
# this code defines a params data structure which contains 
# parameters like learning rate and weight_decay
params = []
for key, value in dict(G.named_parameters()).items():
    if value.requires_grad:
        if 'classifier' not in key:
            params += [{'params': [value], 'lr': args_multi,
                        'weight_decay': 0.0005}]
        else:
            params += [{'params': [value], 'lr': args.multi * 10,
                        'weight_decay': 0.0005}]
# print('params: ' , params)
if "resnet" in args_net:
    F1 = Predictor_deep(num_class=len(class_list), inc=inc)
elif "EfficientNetBase" in args_net:
    F1 = Predictor_deep(num_class=len(class_list), inc=inc)
else:
    F1 = Predictor(num_class=len(class_list), inc=inc,temp=args_T)

weights_init(F1)    # initialize the predictor layer F with random weights
G.cuda()           # load model G into CUDA 
F1.cuda()          # load model G into CUDA 

In [ ]:
# this initializes tensor variables

im_data_s = torch.FloatTensor(1)
im_data_t = torch.FloatTensor(1)
im_data_tu = torch.FloatTensor(1)
gt_labels_s = torch.LongTensor(1)
gt_labels_t = torch.LongTensor(1)
sample_labels_t = torch.LongTensor(1)
sample_labels_s = torch.LongTensor(1)

im_data_s = im_data_s.cuda()
im_data_t = im_data_t.cuda()
im_data_tu = im_data_tu.cuda()
gt_labels_s = gt_labels_s.cuda()
gt_labels_t = gt_labels_t.cuda()
sample_labels_t = sample_labels_t.cuda()
sample_labels_s = sample_labels_s.cuda()

im_data_s = Variable(im_data_s)
im_data_t = Variable(im_data_t)
im_data_tu = Variable(im_data_tu)
gt_labels_s = Variable(gt_labels_s)
gt_labels_t = Variable(gt_labels_t)
sample_labels_t = Variable(sample_labels_t)
sample_labels_s = Variable(sample_labels_s)

print('creating folder: ' , args_checkpath)
if os.path.exists(args_checkpath) == False:
    os.mkdir(args_checkpath)



In [ ]:

def extract_fetures(loader):
    G.eval()
    F1.eval()
    test_loss = 0
    correct = 0
    size = 0
    num_class = len(class_list)
    output_all = np.zeros((0, 512))
    gt_all = np.zeros((0))
    criterion = nn.CrossEntropyLoss().cuda()
    confusion_matrix = torch.zeros(num_class, num_class)
    with torch.no_grad():
        for batch_idx, data_t in enumerate(loader):
            with torch.no_grad():
               im_data_t.resize_(data_t[0].size()).copy_(data_t[0])
               gt_labels_t.resize_(data_t[1].size()).copy_(data_t[1])
            feat = G(im_data_t)
            output1 = F1.fc1(feat)
            output_all = np.r_[output_all, output1.data.cpu().numpy()]
            gt_all = np.r_[gt_all, gt_labels_t.data.cpu().numpy()]
                    
    return output_all, gt_all

def test(loader):
    G.eval()
    F1.eval()
    test_loss = 0
    correct = 0
    size = 0
    num_class = len(class_list)
    output_all = np.zeros((0, num_class))
    criterion = nn.CrossEntropyLoss().cuda()
    confusion_matrix = torch.zeros(num_class, num_class)
    with torch.no_grad():
        for batch_idx, data_t in enumerate(loader):
            with torch.no_grad():
               im_data_t.resize_(data_t[0].size()).copy_(data_t[0])
               gt_labels_t.resize_(data_t[1].size()).copy_(data_t[1])
            feat = G(im_data_t)
            output1 = F1(feat)
            output_all = np.r_[output_all, output1.data.cpu().numpy()]
            size += im_data_t.size(0)
            pred1 = output1.data.max(1)[1]
            for t, p in zip(gt_labels_t.view(-1), pred1.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
            correct += pred1.eq(gt_labels_t.data).cpu().sum()
            test_loss += criterion(output1, gt_labels_t) / len(loader)
    print('\nTest set: Average loss: {:.4f}, '
          'Accuracy: {}/{} F1 ({:.0f}%)\n'.
          format(test_loss, correct, size,
                 100. * correct / size))
    return test_loss.data, 100. * float(correct) / size



In [ ]:
##############################################################
#  extract the deatures from the backbone CNN model
#  Use t-SNE visualization method to visualize the data distribution
#  of the unlabled target samples before we apply SSDAN
################################################# 

out,gt_all= extract_fetures(target_loader_test)
print(out.shape)

from pathlib import Path
from sklearn.datasets import load_digits
from MulticoreTSNE import MulticoreTSNE as TSNE
from matplotlib import pyplot as plt

embeddings = TSNE(n_jobs=4).fit_transform(out)
vis_x = embeddings[:, 0]
vis_y = embeddings[:, 1]
plt.scatter(vis_x, vis_y, c=gt_all, cmap=plt.cm.get_cmap("jet", 12), marker='.')
plt.colorbar(ticks=range(12))
plt.clim(-0.5, 11.5)
plt.xlabel("tSNE dim1")
plt.ylabel("tSNE dim2")
file_name = SSDAN_path+"outputsAndPlots/plot_5001_epoch/"+'tSNE_'+ args_source +'_To_' + args_target +'_before.png'
plt.savefig(file_name  , bbox_inches='tight'  , dpi=300)
plt.show()

In [ ]:
# initilize the model parameters
# recall F1 is predictor model 
# G is feature extractor model using a pretrained CNN

lr = args_lr       # define the learning rate

def zero_grad_all():    # this function makes the gradient zero
    optimizer_g.zero_grad()
    optimizer_f.zero_grad()


G.train()
F1.train()
# recall that "params" contains parameters such as learning rate and weight_decay
optimizer_g = optim.Adam(params,    weight_decay=0.0005)
optimizer_f = optim.Adam(list(F1.parameters()), lr=1.0, weight_decay=0.0005)

param_lr_g = []
for param_group in optimizer_g.param_groups:
    param_lr_g.append(param_group["lr"])
param_lr_f = []
for param_group in optimizer_f.param_groups:
    param_lr_f.append(param_group["lr"])

criterion = nn.CrossEntropyLoss().cuda()
all_step = args_epochs

data_iter_s = iter(source_loader)
data_iter_t = iter(target_loader)
data_iter_t_unl = iter(target_loader_unl)
len_train_source = len(source_loader)
len_train_target = len(target_loader)
len_train_target_semi = len(target_loader_unl)


In [ ]:
# NOW WE START TRAINING

start_time = time.process_time()
best_acc = 0
counter = 0
train_losses = []
entropy_losses = []
tot_loss_test =[]
tot_acc_test =[]
tot_loss_val=[]
tot_acc_val=[]
for step in range(all_step): # epochs 
    # print(step % args_log_interval, (step % args_log_interval) == 0, ' ', end = '')
    # make optimizors 
    optimizer_g = inv_lr_scheduler(param_lr_g, optimizer_g, step,      init_lr=args_lr)
    optimizer_f = inv_lr_scheduler(param_lr_f, optimizer_f, step,   init_lr=args_lr)

    lr = optimizer_f.param_groups[0]['lr']
    if step % len_train_target == 0:
        data_iter_t = iter(target_loader)
    if step % len_train_target_semi == 0:
        data_iter_t_unl = iter(target_loader_unl)
    if step % len_train_source == 0:
        data_iter_s = iter(source_loader)

    data_t = next(data_iter_t)
    data_t_unl = next(data_iter_t_unl)
    data_s = next(data_iter_s)

    # im_data_s.data.resize_(data_s[0].size()).copy_(data_s[0])
    # gt_labels_s.data.resize_(data_s[1].size()).copy_(data_s[1])
    # im_data_t.data.resize_(data_t[0].size()).copy_(data_t[0])
    # gt_labels_t.data.resize_(data_t[1].size()).copy_(data_t[1])
    # im_data_tu.data.resize_(data_t_unl[0].size()).copy_(data_t_unl[0])

    with torch.no_grad():
      im_data_s.resize_(data_s[0].size()).copy_(data_s[0])
      gt_labels_s.resize_(data_s[1].size()).copy_(data_s[1])
      im_data_t.resize_(data_t[0].size()).copy_(data_t[0])
      gt_labels_t.resize_(data_t[1].size()).copy_(data_t[1])
      im_data_tu.resize_(data_t_unl[0].size()).copy_(data_t_unl[0])

    zero_grad_all() # set gardients to zero, this is our fuction above

    data = torch.cat((im_data_s, im_data_t), 0) # concatenate source and target images
    #torch.Size([32, 3, 224, 224])
    target = torch.cat((gt_labels_s, gt_labels_t), 0)  # concatenate source and target labels
    output = G(data)  # apply feature extractor using CNN model
    #print(output)
    #torch.Size([32, 1000])
    
    out1 = F1(output)   # apply class predictor

    loss = criterion(out1, target)  # compute loss function
    loss.backward(retain_graph=True)  # apply a backward pass
    optimizer_g.step()   # optimize G - update weights
    optimizer_f.step()   # optimize F - update weights

    zero_grad_all()  # set gardients to zero, this is our fuction above

    if not args_method == 'S+T':
        output = G(im_data_tu)
        if args_method == 'ENT':
            loss_t = entropy(F1, output, args_lamda)
            loss_t.backward()
            optimizer_f.step()
            optimizer_g.step()
        elif args_method == 'MME':          
            loss_t = adentropy(F1, output, args_lamda)
            loss_t.backward()
            optimizer_f.step()
            optimizer_g.step()
        else:
            raise ValueError('Method cannot be recognized.')

        log_train = 'S {} T {} Train Ep: {} lr{} \t ' \
                    'Loss Classification: {:.6f} Loss T {:.6f} ' \
                    'Method {}\n'.format(args_source, args_target,
                                          step, lr, loss.data,
                                          -loss_t.data, args_method)
    else:
        log_train = 'S {} T {} Train Ep: {} lr{} \t ' \
                    'Loss Classification: {:.6f} Method {}\n'.\
            format(args_source, args_target,
                    step, lr, loss.data,
                    args_method)
    G.zero_grad()
    F1.zero_grad()
    zero_grad_all()
    
    train_losses.append(loss.cpu().detach().numpy()) 
    entropy_losses.append(-loss_t.cpu().detach().numpy())        
    if step % args_log_interval == 0 and step > 0:
        loss_test, acc_test = test(target_loader_test)
        loss_val, acc_val = test(target_loader_val)
        tot_loss_test.append(loss_test)
        tot_acc_test.append(acc_test)
        tot_loss_val.append(loss_val)
        tot_acc_val.append(acc_val)
        G.train()
        F1.train()
        if acc_val >= best_acc:
            best_acc = acc_val
            best_acc_test = acc_test
            counter = 0
        else:
            counter += 1
        if args_early:
            if counter > args_patience:
                break
        print(log_train)
        print('best acc test %f best acc val %f' % (best_acc_test,best_acc))
        # print('outputsAndPlots %s' % outputsAndPlots_file)
        with open(outputsAndPlots_file, 'a') as f:
            f.write(log_train)
            f.write('step %d best test %f validation %f \n' % (step,
                                                      best_acc_test,
                                                      best_acc))
        G.train()
        F1.train()

        if args_save_check:
            print('saving model')
            torch.save(G.state_dict(),
                        os.path.join(args_checkpath,
                                    "G_iter_model_{}_{}_"
                                    "to_{}_step_{}.pth.tar".
                                    format(args_method, args_source,
                                            args_target, step)))
            torch.save(F1.state_dict(),
                        os.path.join(args_checkpath,
                                    "F1_iter_model_{}_{}_"
                                    "to_{}_step_{}.pth.tar".
                                    format(args_method, args_source,
                                            args_target, step)))

end_time = time.process_time()
print()
print('best acc test %f best acc val %f \n' % (best_acc_test,best_acc))
print("Process time: ", end_time - start_time)

with open(outputsAndPlots_file, 'a') as f:
    f.write('best acc test %f best acc val %f \n' % (best_acc_test,best_acc))
    f.write('CPU time: %f \n' % (end_time - start_time ) )

print('saving model')
torch.save(G.state_dict(),
            os.path.join(args_checkpath,
                        "G_iter_model_{}_{}_"
                        "to_{}_step_{}.pth.tar".
                        format(args_method, args_source,
                                args_target, step)))
torch.save(F1.state_dict(),
            os.path.join(args_checkpath,
                        "F1_iter_model_{}_{}_"
                        "to_{}_step_{}.pth.tar".
                        format(args_method, args_source,
                                args_target, step)))


In [ ]:
# Saving and Loading Models with Shapes
# We might want to save the structure of this class together with the model, 
# in which case we can pass model (and not model.state_dict()) to the saving function:

file_name = SSDAN_path+"outputsAndPlots/plot_5001_epoch/models/G_"+ args_source +'_To_' + args_target +'.pth'
file_name1 = SSDAN_path+"outputsAndPlots/plot_5001_epoch/models/F1_'+ args_source +'_To_' + args_target +'.pth'
# torch.save(G,file_name)
# torch.save(F1,file_name1)
torch.save(G.state_dict(),file_name)
torch.save(F1.state_dict(),file_name1)
print("Saved PyTorch Model State to" + file_name)
print("Saved PyTorch Model State to" + file_name1)

In [ ]:
# When loading model weights, we needed to instantiate the model class first, 
# because the class defines the structure of a network.
#We can then load the model like this:
file_name = SSDAN_path+"outputsAndPlots/plot_5001_epoch/models/G_"+ args_source +'_To_' + args_target +'.pth'
file_name1 = SSDAN_path+"outputsAndPlots/plot_5001_epoch/models/F1_"+ args_source +'_To_' + args_target +'.pth'
G = torch.load(file_name)
F1= torch.load(file_name1)

In [ ]:
#We can then load the model like this:
file_name = main_folder_path+"code/trained_models/test/"+ args_source +'_To_' + args_target +'.pth'
file_name1 = main_folder_path+"code/trained_models/test/"+'G_'+ args_source +'_To_' + args_target +'.pth'
F1 = torch.load(file_name)
G = torch.load(file_name1)
G.keys()

In [ ]:
out,gt_all= extract_fetures(target_loader_test)
print(out.shape)

embeddings = TSNE(n_jobs=4).fit_transform(out)
vis_x = embeddings[:, 0]
vis_y = embeddings[:, 1]
plt.scatter(vis_x, vis_y, c=gt_all, cmap=plt.cm.get_cmap("jet", 12), marker='.')
plt.colorbar(ticks=range(12))
plt.clim(-0.5, 11.5)
plt.xlabel("tSNE dim1")
plt.ylabel("tSNE dim2")
file_name = SSDAN_path+"outputsAndPlots/plot_5001_epoch/"+'tSNE_'+ args_source +'_To_' + args_target +'_after.png'
plt.savefig(file_name  , bbox_inches='tight'  , dpi=300)
plt.show()

In [ ]:
d= {'entropy_losses':entropy_losses, 'train_losses':train_losses}
file_name = SSDAN_path+"outputsAndPlots/plot_5001_epoch/"+ 'losses_'+ args_source +'_To_' + args_target +'2.npz'
np.savez(file_name,**d)

In [ ]:
file_name = SSDAN_path+"outputsAndPlots/plot_5001_epoch/"+'losses_'+ args_source +'_To_' + args_target +'2.npz'
r= np.load(file_name)
r.keys()
e1 = r['entropy_losses']
t= np.load(file_name)
t.keys()
tl = t['train_losses']
plt.figure(figsize=(10,5))
plt.title("Training and Validation Loss")
plt.plot(e1,label="val")
plt.plot(tl,label="train")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
file_name_Acc = SSDAN_path+"outputsAndPlots/plot_5001_epoch/"+'Acc_'+ args_source +'_To_' + args_target +'2.npz'
d= {'tot_acc_test':tot_acc_test, 'tot_acc_val':tot_acc_val}
np.savez(file_name_Acc,**d)

In [ ]:
file_name_Acc = SSDAN_path+"outputsAndPlots/plot_5001_epoch/"+'Acc_'+ args_source +'_To_' + args_target +'2.npz'
s= np.load(file_name_Acc)
s.keys()
a = s['tot_acc_test']
k= np.load(file_name_Acc)
k.keys()
a1 = k['tot_acc_val']
plt.figure(figsize=(10,5))
plt.title("Training and Validation ACC")
plt.plot(a,label="Acc")
plt.plot(a1,label="val")
plt.xlabel("iterations")
plt.ylabel("Acc")
plt.legend()
plt.show()